## This script should demonstrate the main functions

In [1]:
# from datetime import datetime
# from sklearn.metrics import accuracy_score, f1_score, recall_score
import os
from openai import OpenAI
from dotenv import load_dotenv
import random
import json
from datetime import datetime, timedelta
from utils.evaluation_utils import evaluate_label_single, calculate_overall_metrics
from utils.generation_utils import generate_prompt, label2email
from utils.labeling_utils import re_analyze_email, gpt_label, gpt_label_eg, gpt_label_ft
from utils.gpt_utils import gpt_completion

In [2]:
# Load the environment variables from the .env file
# In this .env, it contains openai's API Key.
load_dotenv()
# Load the key to call the client.
client = OpenAI()
model_name="gpt-4o-mini"
# model_name="gpt-4o"

In [3]:
test_data_path = "./data/PAIRED_test.json"
with open(test_data_path, 'r') as file:
    test_data = json.load(file)

train_data_path = "./data/PAIRED_train.json"
with open(train_data_path, 'r') as file:
    train_data = json.load(file)

## 1. If you want to write an email
Let's say that you want to held a study session for CS6320, and you write an email to all of your team:

In [8]:
label_studysession = {
    "Spam":"No",
    "Subject": "Study Session for NLP 6320 Final Exam",
    "Sender": "Edward",
    "send_date": "2024-12-06",
    "Time_Sensitive": "Yes",
    "Start": "2024-12-06 13:30",
    "End": "2024-12-06 15:30",
    "Type": "Event",
    "Category": "Study",
    "Format": "In-person",
    "Location": "Library",
    "Action_Required": "Yes",
    "Priority_Level": "High"
}

In [9]:
email = label2email(label_studysession, client, temperature=1.0, model=model_name)
print(email)

From: Edward  
Sent: 2024-12-06  
Subject: Study Session for NLP 6320 Final Exam  

Dear Classmates,

I hope you are all doing well! I’m organizing a study session for our NLP 6320 final exam today, December 6, 2024, from 1:30 PM to 3:30 PM in the library. This will be a great opportunity for us to review key topics, share insights, and clarify any doubts as we prepare for the exam.

Please let me know if you can make it, so I can arrange adequate space and materials for everyone. Your participation would be highly beneficial for all of us!

Looking forward to hearing from you soon!

Best,  
Edward


## 2. If you received an email and you want to add it to your calendar:

In [14]:
email_received = """
From: Edward  
Date: 2024-12-03  
Subject: Office Hour of CS1234

Hi all,

According to the requirements by some students, I'll held a special office-hour today, starting from 1PM to 2PM.
We will talk about final exams, and I will answer your questions regarding the exam.
I'll send an MS Teams link for this later.

Regards,
Edward

"""

In [15]:
label = gpt_label_eg(client, email_received, train_data, temperature=0.7, model=model_name)
print(label)

{
    "Spam": "No",
    "Subject": "Office Hour of CS1234",
    "Sender": "Edward",
    "send_date": "2024-12-03",
    "Time_Sensitive": "Yes",
    "Start": "2024-12-03 13:00",
    "End": "2024-12-03 14:00",
    "Type": "Event",
    "Category": "Study",
    "Format": "Online",
    "Location": "MS Teams (link to be sent later)",
    "Action_Required": "No",
    "Priority_Level": "Medium"
}
